The cell below is the VaR-CVaR algorithm for a three-asset case and prints out VaR and CVaR estimates and the optimal weights of each asset such that CVaR is minimized.

It runs for about 2.5 min.

In [ ]:
import numpy as np

###############################################################################
# SGLD FUNCTION: MULTI-ASSET VaR-CVaR USING SGLD
###############################################################################

def g(w):
    exp_w = np.exp(w)
    return exp_w / np.sum(exp_w)

def sgld_var_cvar(dist_sampler, q, N, gamma, beta, lam, collect, random_seed=None):
    """
    Perform SGLD for 3 assets to minimize V(theta, w) = E[ (1/(1-q)) * ( g(w)^T X - theta )_+ + theta ] + gamma * (theta^2 + w^2).
    
    Parameters:
        dist_sampler: function that returns a sample 3-dim vector X.
        q: confidence level.
        N: total number of iterations.
        gamma: positive constant in the theta^2 penalization term.
        beta: (positive) inverse-temperature used in the SGLD noise.
        lam: (positive) step size.
        collect: number of samples to collect at the end of the iterations.
        random_seed: to fix the seed for reproducibility.
    
    Returns:
        final_thetas: last sampled theta values.
        final_ws: last sampled w vectors.
        final_Xs: corresponding samples of X vectors.
    """
    if random_seed is not None:
        np.random.seed(random_seed)

    # Initial values
    theta = 0.0
    w = np.zeros(3)
    
    thetas = np.zeros(N)
    ws = np.zeros((N, 3))
    Xs = np.zeros((N, 3))

    # Main SGLD loop
    for n in range(N):

        # Sample X_{n+1} from the chosen distribution
        x = dist_sampler()

        gw = g(w)
        z = np.dot(gw, x) # g(w)^T X

        # Compute H_theta
        # H(theta, x) = 1 + 2 * gamma * theta    if z < theta,
        #             = 1 - 1/(1-q) + 2 * gamma * theta  if z >= theta.
        if z < theta:
            H_theta = 1.0 + 2 * gamma * theta
        else:
            H_theta = 1.0 - 1.0/(1.0 - q) + 2 * gamma * theta

        # Compute H_w
        # H(w, x) = 2 * gamma * theta    if z < theta,
        #         = 1/(1-q) * (g hat) + 2 * gamma * theta  if z >= theta.
        H_w = np.zeros(3)
        if z >= theta:
            for j in range(3):
                g_hat_j = 0.0
                for i in range(3):
                    if i == j:
                        dgi_dwj = gw[i] * (1 - gw[j])
                    else:
                        dgi_dwj = -gw[i] * gw[j]
                    g_hat_j += dgi_dwj * x[i]
                H_w[j] = (1.0 / (1.0 - q)) * g_hat_j + 2 * gamma * w[j]
        else:
            H_w = 2 * gamma * w

        # Update rule:
        # theta_{n+1} = theta_n - lambda * H + sqrt(2*lambda/beta)*N(0,1)
        # w_{n+1} = w_n - lambda * H + sqrt(2*lambda/beta)*N(0,I_3)
        theta = theta - lam * H_theta + np.random.normal(scale=np.sqrt(2.0 * lam / beta))
        w = w - lam * H_w + np.random.normal(scale=np.sqrt(2.0 * lam / beta), size=3)

        thetas[n] = theta
        ws[n, :] = w
        Xs[n, :] = x

    return thetas[-collect:], ws[-collect:, :], Xs[-collect:, :]

###############################################################################
# FUNCTION TO ESTIMATE VaR AND CVaR FROM THE SGLD ALGORITHM
###############################################################################

def estimate_var_cvar(thetas, ws, xs, q, gamma):
    """
    Given arrays of theta, w and x, estimate:
        VaR_SGLD = average of theta,
        CVaR_SGLD = average of [ theta + (1/(1-q)) * (g(w)^T x - theta)_+ ] + gamma * (theta^2 + w^2).
    """
    gws = np.exp(ws - np.max(ws, axis=1, keepdims=True))
    gws = gws / np.sum(gws, axis=1, keepdims=True)

    zs = np.sum(gws * xs, axis=1)

    var_est = np.mean(thetas)

    cvar_terms = thetas + (1.0 / (1.0 - q)) * np.maximum(0.0, zs - thetas)
    cvar_est = np.mean(cvar_terms + gamma * (thetas**2 + np.sum(ws**2, axis=1)))


    return var_est, cvar_est

###############################################################################
# FUNCTIONS TO GENERATE REFERENCE VALUES
############################################################################### 

def generate_simplex_grid(m):
    """
    Generate evenly spaced 3D weight vectors (w1, w2, w3) on the 2-simplex:
    w1 + w2 + w3 = 1 and w_i >= 0.
    """
    grid = []
    for i in range(m + 1):
        for j in range(m + 1 - i):
            k = m - i - j
            w1 = i / m
            w2 = j / m
            w3 = k / m
            grid.append([w1, w2, w3])
    return np.array(grid)

def simulate_reference_values(mus, variances, q, N, m, random_seed=42):

    np.random.seed(random_seed)
    
    # Distribution parameters
    mu = np.array(mus)
    cov = np.diag(variances)

    # Generate samples from multivariate normal
    X = np.random.multivariate_normal(mu, cov, size=N)  # shape (num_samples, 3)

    # Generate candidate weights
    weights = generate_simplex_grid(m)

    # Compute VaR and CVaR for each weight vector
    vars = []
    cvars = []

    for w in weights:
        Z = X @ w  # portfolio return
        var = np.quantile(Z, q)
        vars.append(var)

        cvar = np.mean(Z[Z >= var])
        cvars.append(cvar)

    # Take the minimum CVaR as the optimal CVaR
    cvars = np.array(cvars)
    best_idx = np.argmin(cvars)
    
    # Return corresponding optimal w, VaR and CVaR
    return weights[best_idx], vars[best_idx], cvars[best_idx]


###############################################################################
# EXPERIMENTS: PRINT RESULTS FOR THE 5 DIFFERENT CASES
###############################################################################

def run_normal_experiments(mus, variances):
    """
    Run 3-asset SGLD experiments for the Normal distribution case using independent simulations.
    
    For the given parameter sets (mu, var), we run 2 independent SGLD simulations, each with 10^5 iterations.
    From each simulation, we take the last 1000 theta and corresponding x.
    The VaR and CVaR for the given confidence level q (via SGLD) are computed
    and then averaged over the simulations.
    
    Parameters:
        mus: list of 3 means, one for each asset
        variances: list of 3 variances, one for each asset
    """
    num_simulations = 5
    N = int(1e5)
    gamma = 1e-8
    beta = 1e8
    lam = 1e-4
    collect = 1000
    q = 0.95

    mu = np.array(mus)
    cov = np.diag(variances)

    def multivariate_sampler():
        return np.random.multivariate_normal(mu, cov)

    weights = np.ones((num_simulations,3))/3
    vars = []
    cvars = []

    for i in range(num_simulations):
        seed = 42 + i
        thetas, ws, xs = sgld_var_cvar(multivariate_sampler, q, N, gamma, beta, lam, collect, random_seed=seed)
        var, cvar = estimate_var_cvar(thetas, ws, xs, q, gamma)
        vars.append(var)
        cvars.append(cvar)

        gws = np.exp(ws - np.max(ws, axis=1, keepdims=True))
        gws = gws / np.sum(gws, axis=1, keepdims=True)
        weights[i, :] = np.mean(gws, axis=0)


    var_avg = np.mean(vars)
    cvar_avg = np.mean(cvars)
    w_avg = np.mean(weights, axis=0)

    w_opt, var_opt, cvar_opt = simulate_reference_values(mus, variances, q, N, 100, random_seed=seed)

    print(f"{'N(' + str(mus[0]) + ', ' + str(variances[0]) + ')':<20}",
          f"{'N(' + str(mus[1]) + ', ' + str(variances[1]) + ')':<20}",
          f"{'N(' + str(mus[2]) + ', ' + str(variances[2]) + ')':<20}",
          f"{w_avg[0]:7.4f} {w_avg[1]:7.4f} {w_avg[2]:7.4f}",
          f"{var_avg:8.4f} {cvar_avg:8.4f}",
          f"{w_opt[0]:7.3f} {w_opt[1]:7.3f} {w_opt[2]:7.3f}",
          f"{var_opt:8.4f} {cvar_opt:8.4f}")




###############################################################################
# RUN EXPERIMENTS
###############################################################################

# Normal parameters for 5 cases
mus = [[500, 0, 0], [500, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 2]]
variances = [[1, 1e6, 1e-4], [1, 1e6, 1], [1e3, 1, 4], [1, 4, 1e-4], [1, 4, 1]]

print("95% VaR and CVaR Estimates using SGLD for the case with 3 assets:\n")
print(f"{'X1':<20} {'X2':<20} {'X3':<20} "
      f"{'w1':>5} {'w2':>7} {'w3':>7} "
      f"{'VaR':>8} {'CVaR':>9} "
      f"{'w1*':>7} {'w2*':>7} {'w3*':>7} "
      f"{'VaR*':>8} {'CVaR*':>8}")
print("-" * 150)

for i in range(5):
    run_normal_experiments(mus[i], variances[i])
    print("\n")


95% VaR and CVaR Estimates using SGLD for the case with 3 assets:

X1                   X2                   X3                      w1      w2      w3      VaR      CVaR     w1*     w2*     w3*     VaR*    CVaR*
------------------------------------------------------------------------------------------------------------------------------------------------------
N(500, 1)            N(0, 1000000.0)      N(0, 0.0001)          0.0001  0.0000  0.9999   0.0646   0.0724   0.000   0.000   1.000   0.0165   0.0206


N(500, 1)            N(0, 1000000.0)      N(0, 1)               0.0001  0.0001  0.9999   1.6831   2.0863   0.000   0.000   1.000   1.6506   2.0702


N(0, 1000.0)         N(0, 1)              N(0, 4)               0.0040  0.8036  0.1924   1.4885   1.8861   0.000   0.800   0.200   1.4758   1.8487


N(0, 1)              N(1, 4)              N(0, 0.0001)          0.0303  0.0123  0.9574   0.0790   0.0926   0.000   0.000   1.000   0.0165   0.0206


N(0, 1)              N(1, 4)            